<a href="https://colab.research.google.com/github/ieesha1102/Art-Gallery-Management/blob/main/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


In [2]:
!pip install jellyfish

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.5 MB/s eta 0:00:00


In [3]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 2s (2,830 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [4]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import files
import pandas as pd
import numpy as np
import sys
import warnings
import requests
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import easyocr
import nltk
import jellyfish
import pytesseract

In [6]:
warnings.filterwarnings('ignore')

In [7]:
def open_image(url):
  #https://drive.google.com/open?id=1ZdOD8MHMct93qywCiK-4Dbc2X_zRWrXq
  #modify link
  marker = url.find('id=') + 3
  img_url = 'https://docs.google.com/uc?export=download&id='+ url[ marker :  ]
  print(url[ marker :  ])
  print(img_url)
  #open image
  img = Image.open(requests.get(img_url, stream = True).raw)
  #convert image to an array of bits
  numpy_image=np.array(img) 
  #load image to cv2 so that it's easy to process
  img = cv2.cvtColor(numpy_image, cv2.COLOR_BGR2RGB)
  #store height, weight and number of channels
  (h, w, c) = img.shape[:3]
  print("width: {} pixels".format(w))
  print("height: {}  pixels".format(h))
  print("channels: {}".format(c))
  #open image on screen
  return img

In [8]:
def rotate_image(img):
  rotated_image = np.rot90(img, k=1)
  return rotated_image

In [9]:
def crop_image(img):
  #perform precropping
  img = img[30:img.shape[0]-30, 30:img.shape[1]-30]
  #convert image from color to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  #gray-scale to black&white
  gray = 255*(gray < 50).astype(np.uint8)
  gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, np.ones( (2, 2), dtype=np.uint8)) 
  #find non-zero ie not white bits
  coords = cv2.findNonZero(gray) 
  #get the bounding rectangle
  x, y, w, h = cv2.boundingRect(coords)
  #crop image
  rect = img[y:y+h, x:x+w]
  #show images
  return rect

In [10]:
def detect_text(img):
  reader = easyocr.Reader(['en'])
  result = reader.readtext(img, detail = 0)
  res = ' '.join(result)
  return res

In [11]:
def calc_similarity(text):
  # Jaro-Winkler distance metric
  sentence = "Well begun is half done"
  text = "".join(text.split())
  sentence = "".join(sentence.split())
  jw_distance = jellyfish.jaro_winkler(text, sentence)
  return jw_distance

In [12]:
def calc_azimuth_angle(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Apply binary threshold to the image
  _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  # Apply morphological opening to remove noise
  kernel = np.ones((3,3), np.uint8)
  opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
  # Detect lines using the Hough transform
  lines = cv2.HoughLines(opening, rho=1, theta=np.pi/180, threshold=100)
  # Calculate the orientation of each line
  orientations = []
  for line in lines:
    rho, theta = line[0]
    slope = np.tan(theta)
    orientation = np.arctan(slope) * 180 / np.pi
    orientations.append(orientation)
  # Find the dominant orientation
  bins = np.arange(-90, 91, 5)
  hist, _ = np.histogram(orientations, bins=bins)
  dominant_orientation = bins[np.argmax(hist)]
  # Calculate the azimuth angle
  azimuth_angle = 90 - dominant_orientation
  if azimuth_angle < 0:
    azimuth_angle += 180
  return azimuth_angle

In [13]:
def calc_baseline(img):
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Apply a binary threshold to the image
  _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
  # Apply the Hough Transform to detect lines in the image
  lines = cv2.HoughLinesP(thresh, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)
  # Calculate the slope of the detected lines
  slopes = []
  for line in lines:
      x1, y1, x2, y2 = line[0]
      if x2 - x1 == 0:
          slope = 1000000 # Avoid divide by zero error
      else:
          slope = (y2 - y1) / (x2 - x1)
      slopes.append(slope)
  # Calculate the average slope of the baseline
  avg_slope = np.mean(slopes)
  # Calculate the angle of the baseline
  angle = np.arctan(avg_slope) * 180 / np.pi
  # Print the angle of the baseline
  return angle

In [14]:
def calc_slant(img):
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Threshold the image to get a binary image
  _, binary_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
  # Apply edge detection to the binary image
  edges = cv2.Canny(binary_img, 50, 150, apertureSize=3)
  # Apply the Hough transform to detect lines in the image
  lines = cv2.HoughLines(edges, rho=1, theta=np.pi/180, threshold=100)
  # Calculate the average angle of the detected lines
  angle_sum = 0
  num_lines = len(lines)
  for i in range(num_lines):
      rho, theta = lines[i][0]
      angle_sum += theta
  if num_lines > 0:
      avg_angle = angle_sum/num_lines
  else:
      avg_angle = 0
  # Calculate the slant angle of letters by subtracting the average angle from 90 degrees
  slant_angle = np.rad2deg(np.pi/2 - avg_angle)
  return slant_angle

In [15]:
def calc_space_words(img):
  grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  (thresh, im_bw) = cv2.threshold(grayImage, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  thresh = 127
  im_bw = cv2.threshold(grayImage, thresh, 255, cv2.THRESH_BINARY)[1]
  hieght = im_bw.shape[0]
  width = im_bw.shape[1]
  black = 0
  white = 0
  #count black and white pixels
  for j in range(0,width):
    if(im_bw[round(hieght/2)][j] == 0):
        white = white+1
    else:
        black = black+1
  return white/4

In [16]:
#1 if dot is above i else 0
def i_dot_position(img):
  pos = 0
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Apply thresholding to the grayscale image
  _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
  # Erode the image to remove noise and smoothen it
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  eroded = cv2.erode(thresh, kernel, iterations=1)
  # Find contours in the eroded image
  contours, _ = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  # Iterate through the contours and get the bounding boxes
  for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    # Check if the bounding box is above the letter "i" at the 9th position
    if 8 < x < 10 and h < w:
        pos = 1
        break
  return pos

In [17]:
#1-loop present 0-loop not present
def detect_loop(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = 255*(gray < 50).astype(np.uint8)
  gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, np.ones( (2, 2), dtype=np.uint8))
  coords = cv2.findNonZero(gray)
  x, y, w, h = cv2.boundingRect(coords)
  rect = img[y:y+h, x:x+w]
  image = cv2.medianBlur(rect,5)
  # Load the image and convert to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # Apply thresholding to convert to binary image
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  # Use Tesseract OCR to extract text from the image
  text = pytesseract.image_to_string(thresh)
  index = text.find('g')
  # Extract the letter 'a' from the original image
  boxes = pytesseract.image_to_boxes(image)
  for b in boxes.splitlines():
    b = b.split(' ')
    x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
    cv2.rectangle(image, (x, image.shape[0] - y), (w, image.shape[0] - h), (0, 0, 255), 2)
    if b[0]=='g':
        #x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
        letter_image = image[int(y):int(h), int(x):int(w)]
        break
  letter_image = image[int(h*0.8):int(h), int(x):int(w)]
  # Convert the lower region of the letter to grayscale
  gray = cv2.cvtColor(letter_image, cv2.COLOR_BGR2GRAY)
  # Apply thresholding to isolate the shape of the lower region of the letter
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  # Apply edge detection to highlight the contours of the lower region of the letter
  edges = cv2.Canny(thresh, 100, 200)
  # Use contour detection to identify any loops in the lower region of the letter
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  # Draw the contours on the original image
  cv2.drawContours(letter_image, contours, -1, (0, 0, 255), 2)
  has_loop = False
  for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w < h:
        has_loop = True
        break
  if has_loop:
    return 1
  else:
    return 0

In [18]:
def calc_size_letters(img):
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Apply a binary threshold to the image
  _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
  # Calculate the size of the letters
  sizes = []
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  for cnt in contours:
      x, y, w, h = cv2.boundingRect(cnt)
      size = w * h
      sizes.append(size)
  # Calculate the average size of the letters
  average_size = np.mean(sizes)
  return average_size

In [19]:
def calc_zone_1(img):
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Threshold the image
  ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  # Find contours in the image
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  # Get the bounding box for each contour
  boxes = [cv2.boundingRect(cnt) for cnt in contours]
  # Get the three zones by dividing the image horizontally into three equal parts
  height, width = img.shape[:2]
  zone_height = height // 3
  zone1 = img[0:zone_height, 0:width]
  # Compute the average pixel value for each zone
  zone1_avg = np.mean(zone1)
  return zone1_avg

In [20]:
def calc_zone_2(img):
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Threshold the image
  ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  # Find contours in the image
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  # Get the bounding box for each contour
  boxes = [cv2.boundingRect(cnt) for cnt in contours]
  # Get the three zones by dividing the image horizontally into three equal parts
  height, width = img.shape[:2]
  zone_height = height // 3
  zone2 = img[zone_height:zone_height*2, 0:width]
  # Compute the average pixel value for each zone
  zone2_avg = np.mean(zone2)
  return zone2_avg

In [21]:
def calc_zone_3(img):
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Threshold the image
  ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  # Find contours in the image
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  # Get the bounding box for each contour
  boxes = [cv2.boundingRect(cnt) for cnt in contours]
  # Get the three zones by dividing the image horizontally into three equal parts
  height, width = img.shape[:2]
  zone_height = height // 3
  zone3 = img[zone_height*2:height, 0:width]
  # Compute the average pixel value for each zone
  zone3_avg = np.mean(zone3)
  return zone3_avg

In [22]:
def calc_loop_sharpness(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = 255*(gray < 50).astype(np.uint8)
  gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, np.ones( (2, 2), dtype=np.uint8))
  coords = cv2.findNonZero(gray)
  x, y, w, h = cv2.boundingRect(coords)
  rect = img[y:y+h, x:x+w]
  image = cv2.medianBlur(rect,5)
  # Load the image and convert to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # Apply thresholding to convert to binary image
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  # Use Tesseract OCR to extract text from the image
  text = pytesseract.image_to_string(thresh)
  index = text.find('g')
  # Extract the letter 'a' from the original image
  boxes = pytesseract.image_to_boxes(image)
  for b in boxes.splitlines():
      b = b.split(' ')
      x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
      cv2.rectangle(image, (x, image.shape[0] - y), (w, image.shape[0] - h), (0, 0, 255), 2)
      if b[0]=='g':
          #x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
          letter_image = image[int(y):int(h), int(x):int(w)]
          break
  letter_image = image[int(h*0.8):int(h), int(x):int(w)]
  # Convert the lower region of the letter to grayscale
  gray = cv2.cvtColor(letter_image, cv2.COLOR_BGR2GRAY)
  # Apply thresholding to isolate the shape of the lower region of the letter
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  # Apply edge detection to highlight the contours of the lower region of the letter
  edges = cv2.Canny(thresh, 100, 200)
  # Use contour detection to identify any loops in the lower region of the letter
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  # Draw the contours on the original image
  cv2.drawContours(letter_image, contours, -1, (0, 0, 255), 2)
  has_loop = False
  for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      if w < h:
          has_loop = True
          break
  if has_loop:
      sharpness_values = []
      for contour in contours:
          # Compute the gradient magnitude of the image in the vicinity of the contour
          x, y, w, h = cv2.boundingRect(contour)
          gradient = cv2.Sobel(gray[y:y + h, x:x + w], cv2.CV_64F, 1, 1, ksize=3)
          gradient_mag = np.sqrt(gradient ** 2)
          # Measure the local contrast of the gradient
          sharpness = cv2.meanStdDev(gradient_mag)[1][0][0]
          sharpness_values.append(sharpness)
      # Calculate the average loop sharpness value
      if len(sharpness_values) > 0:
          avg_sharpness = sum(sharpness_values) / len(sharpness_values)
          return avg_sharpness
  else:
      return 0

In [23]:
def characteristics(url):
  img = open_image(url)
  print("Orignal Image:")
  # cv2_imshow(img)
  baseline = calc_baseline(img)
  print("Baseline:",baseline)
  if baseline > 80:
    img = rotate_image(img)
    # cv2_imshow(img)
  text = detect_text(img)
  print("Text:", text)
  similarity = calc_similarity(text)
  print("Similarity:",similarity)
  crop = crop_image(img)
  print("Cropped image")
  # cv2_imshow(crop)
  slant = calc_slant(crop)
  print("Slant:",slant)
  azimuth_angle = calc_azimuth_angle(crop)
  print("Azimuth angle:",azimuth_angle)
  space_words = calc_space_words(crop)
  print("Space between words:", space_words)
  i_dot = i_dot_position(crop)
  print("i-dot position:",i_dot)
  loop = detect_loop(crop)
  print("Loop:",loop)
  size_letters = calc_size_letters(crop)
  print("Average size of letters:",size_letters)
  zone_1 = calc_zone_1(crop)
  print("Zone 1:",zone_1)
  zone_2 = calc_zone_2(crop)
  print("Zone 2:",zone_2)
  zone_3 = calc_zone_3(crop)
  print("Zone 3:", zone_3)
  loop_sharpness = calc_loop_sharpness(crop)
  print("Loop Sharpness:", loop_sharpness)

In [24]:
characteristics('https://drive.google.com/open?id=1geOe70z-7y_Ji5ZABvENdpWf-chnciYD')

1geOe70z-7y_Ji5ZABvENdpWf-chnciYD
https://docs.google.com/uc?export=download&id=1geOe70z-7y_Ji5ZABvENdpWf-chnciYD
width: 3468 pixels
height: 4624  pixels
channels: 3
Orignal Image:


Baseline: 89.99915578230033
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteText: iS kay dcno Lezun wel
Similarity: 0.4505381099808344
Cropped image
Slant: 9.949999707626079
Azimuth angle: 45
Space between words: 271.0
i-dot position: 0
Loop: 1
Average size of letters: 81.82716413508332
Zone 1: 130.23519242730194
Zone 2: 117.68563334440951
Zone 3: 118.20190751552501
Loop Sharpness: 3.6739028040143387


In [25]:
uploaded = files.upload()

Saving Assessment.xlsx to Assessment.xlsx


In [27]:
responses = pd.read_excel('Assessment.xlsx')
responses.shape

(88, 28)

In [28]:
responses.head()

,Timestamp,Score,Acknowledgment,Age,Gender,"I found it hard to wind down ( To diminish gradually in energy, intensity, or scope)",I was aware of dryness of my mouth,I couldn’t seem to experience any positive feeling at all,"I experienced breathing difficulty (eg, excessively rapid breathing, breathlessness in the absence of physical exertion)",I found it difficult to work up the initiative to do things,...,I was intolerant of anything that kept me from getting on with what I was doing,I felt I was close to panic,I was unable to become enthusiastic about anything,I felt I wasn’t worth much as a person,I felt that I was rather touchy,"I was aware of the action of my heart in the absence of physicalexertion (eg, sense of heart rate increase, heart missing a beat)",I felt scared without any good reason,I felt that life was meaningless,Upload Photo,Email Address
0,2022-12-12 13:13:48.749,0,"Yes, I Agree",18,Female,0,0,0,0,2,...,1,1,0,1,1,0,0,1,https://drive.google.com/open?id=1Xl0_7Y-S3MDV...,NaN
1,2022-12-12 13:13:58.670,0,"Yes, I Agree",19,Female,2,2,2,2,0,...,1,3,3,3,2,3,2,3,https://drive.google.com/open?id=1zS9I7xcW0heE...,NaN
2,2022-12-12 13:14:53.896,0,"Yes, I Agree",17,Female,0,1,0,0,0,...,0,1,0,1,1,2,0,1,https://drive.google.com/open?id=1ZdOD8MHMct93...,NaN
3,2022-12-12 13:15:16.523,0,"Yes, I Agree",18,Female,1,0,1,0,1,...,3,3,1,1,1,1,1,0,https://drive.google.com/open?id=1jFa4hoDh5_NR...,NaN
4,2022-12-12 13:15:43.242,0,"Yes, I Agree",21,Female,1,1,1,1,0,...,3,2,1,0,3,2,1,0,https://drive.google.com/open?id=1t8hdjGT4m-y-...,NaN


In [29]:
df = pd.DataFrame(columns = ['age','gender','depression_score', 'anxiety_score', 'stress_score','depression','anxiety','stress','image'])

In [30]:
for i in responses.index:
    age = responses.iloc[i][3]
    gender = responses.iloc[i][4]
    depression_score = responses.iloc[i][7] + responses.iloc[i][9] + responses.iloc[i][14] + responses.iloc[i][17] + responses.iloc[i][20] 
    + responses.iloc[i][21] + responses.iloc[i][25]
    anxiety_score = responses.iloc[i][6] + responses.iloc[i][8] + responses.iloc[i][11] + responses.iloc[i][13] + responses.iloc[i][19] 
    + responses.iloc[i][23] + responses.iloc[i][24]
    stress_score = responses.iloc[i][5] + responses.iloc[i][10] + responses.iloc[i][12] + responses.iloc[i][15] + responses.iloc[i][16] 
    + responses.iloc[i][18] + responses.iloc[i][22]
    image = responses.iloc[i][26]
    depression_score = depression_score*2
    anxiety_score = anxiety_score*2
    stress_score = stress_score*2
    
    if(depression_score>=0 and depression_score<=9):
        depression = "normal"
    elif(depression_score>=10 and depression_score<=13):
        depression = "mild"
    elif(depression_score>=14 and depression_score<=20):
        depression ="moderate"
    elif(depression_score>=21 and depression_score<=27):
        depression = "severe"
    else:
        depression="extremely severe"
    if(anxiety_score>=0 and anxiety_score<=7):
        anxiety = "normal"
    elif(anxiety_score>=8 and anxiety_score<=9):
        anxiety = "mild"
    elif(anxiety_score>=10 and anxiety_score<=14):
        anxiety = "moderate"
    elif(anxiety_score>=15 and anxiety_score<=19):
        anxiety = "severe"
    else:
        anxiety = "extremely severe"
    if(stress_score>=0 and stress_score<=14):
        stress = "normal"
    elif(stress_score>=15 and stress_score<=18):
        stress = "mild"
    elif(stress_score>=19 and stress_score<=25):
        stress = "moderate"
    elif(stress_score>=26 and stress_score<=33):
        stress = "severe"
    else:
        stress = "extremely severe"  
    d = pd.DataFrame({
                      'age':[age],
                      'gender':[gender],
                      'depression_score':[depression_score], 
                      'anxiety_score':[anxiety_score],
                      'stress_score':[stress_score],
                      'depression':[depression],
                      'anxiety':[anxiety],
                      'stress':[stress],
                      'image':[image]
                      })
    
    df = df.append(d)

In [31]:
df.head()

,age,gender,depression_score,anxiety_score,stress_score,depression,anxiety,stress,image
0,18,Female,6,6,12,normal,normal,normal,https://drive.google.com/open?id=1Xl0_7Y-S3MDV...
0,19,Female,20,24,22,moderate,extremely severe,moderate,https://drive.google.com/open?id=1zS9I7xcW0heE...
0,17,Female,0,8,4,normal,mild,normal,https://drive.google.com/open?id=1ZdOD8MHMct93...
0,18,Female,8,18,14,normal,severe,normal,https://drive.google.com/open?id=1jFa4hoDh5_NR...
0,21,Female,6,12,8,normal,moderate,normal,https://drive.google.com/open?id=1t8hdjGT4m-y-...


In [32]:
data = pd.DataFrame(columns = ['Depression','Anxiety','Stress','Age','Gender','Azimuth Angle','Baseline','Space between Words','I-dot position','Slant','Loop',
                               'Average Size of Letters','Zone 1','Zone 2','Zone 3','Loop Sharpness'])

In [33]:
data.head()

,Depression,Anxiety,Stress,Age,Gender,Azimuth Angle,Baseline,Space between Words,I-dot position,Slant,Loop,Average Size of Letters,Zone 1,Zone 2,Zone 3,Loop Sharpness


In [34]:
for i in range(len(df)):
  age = df.iloc[i][0]
  gender = df.iloc[i][1]
  depression = df.iloc[i][5]
  anxiety = df.iloc[i][6]
  stress = df.iloc[i][7]
  print(depression, anxiety, stress)
  img = open_image(df.iloc[i][8])
  baseline = calc_baseline(img)
  if baseline > 80:
    img = rotate_image(img)
    baseline = baseline - 90
  text = detect_text(img)
  similarity = calc_similarity(text)
  if similarity > 0.2:
    crop = crop_image(img)
    slant = calc_slant(crop)
    azimuth_angle = calc_azimuth_angle(crop)
    space_words = calc_space_words(crop)
    i_dot = i_dot_position(crop)
    loop = detect_loop(crop)
    size_letters = calc_size_letters(crop)
    zone_1 = calc_zone_1(crop)
    zone_2 = calc_zone_2(crop)
    zone_3 = calc_zone_3(crop)
    loop_sharpness = calc_loop_sharpness(crop)
    d = pd.DataFrame({ 
                      'Depression':[depression], 
                      'Anxiety':[anxiety],
                      'Stress':[stress],
                      'Age':[age],
                      'Gender':[gender],
                      'Azimuth Angle':[azimuth_angle],
                      'Baseline':[baseline],
                      'Space between Words':[space_words],
                      'I-dot position':[i_dot],
                      'Slant':[slant],
                      'Loop':[loop],
                      'Average Size of Letters':[size_letters],
                      'Zone 1':[zone_1],
                      'Zone 2':[zone_2],
                      'Zone 3':[zone_3],
                      'Loop Sharpness':[loop_sharpness]
                      })
    data = data.append(d, ignore_index = True)

normal normal normal
1Xl0_7Y-S3MDV7tIUFlLqVZkWzkoSvpJA
https://docs.google.com/uc?export=download&id=1Xl0_7Y-S3MDV7tIUFlLqVZkWzkoSvpJA
width: 4160 pixels
height: 3120  pixels
channels: 3


moderate extremely severe moderate
1zS9I7xcW0heE-qUYxRTchv-dGhc1bZIU
https://docs.google.com/uc?export=download&id=1zS9I7xcW0heE-qUYxRTchv-dGhc1bZIU
width: 4080 pixels
height: 3072  pixels
channels: 3


normal mild normal
1ZdOD8MHMct93qywCiK-4Dbc2X_zRWrXq
https://docs.google.com/uc?export=download&id=1ZdOD8MHMct93qywCiK-4Dbc2X_zRWrXq
width: 4000 pixels
height: 1231  pixels
channels: 3


normal severe normal
1jFa4hoDh5_NRSQVSzoKnKsZlPEqHvDgk
https://docs.google.com/uc?export=download&id=1jFa4hoDh5_NRSQVSzoKnKsZlPEqHvDgk


width: 368 pixels
height: 361  pixels
channels: 3
normal moderate normal
1t8hdjGT4m-y-HXC68481p1TLTeFM7LtE
https://docs.google.com/uc?export=download&id=1t8hdjGT4m-y-HXC68481p1TLTeFM7LtE
width: 4032 pixels
height: 3024  pixels
channels: 3


...

extremely severe extremely severe severe
1L_d6O-IB-vQbRJRM6xSZUxG7Wx2i7QvP
https://docs.google.com/uc?export=download&id=1L_d6O-IB-vQbRJRM6xSZUxG7Wx2i7QvP
width: 4624 pixels
height: 3468  pixels
channels: 3


moderate severe mild
1o3nl0Zh_pF9VC9o-tDWJZH0UoCuivLlm
https://docs.google.com/uc?export=download&id=1o3nl0Zh_pF9VC9o-tDWJZH0UoCuivLlm
width: 2705 pixels
height: 1179  pixels
channels: 3


moderate severe normal
1WubpVwnWRG8XwcDiBY3B2ENDon-zrj4U
https://docs.google.com/uc?export=download&id=1WubpVwnWRG8XwcDiBY3B2ENDon-zrj4U
width: 4080 pixels
height: 1836  pixels
channels: 3


normal normal normal
1uv15YL4goa3yRuW4XGsg1dk1CevGzOu9
https://docs.google.com/uc?export=download&id=1uv15YL4goa3yRuW4XGsg1dk1CevGzOu9
width: 4624 pixels
height: 2196  pixels
channels: 3


normal mild normal
1geOe70z-7y_Ji5ZABvENdpWf-chnciYD
https://docs.google.com/uc?export=download&id=1geOe70z-7y_Ji5ZABvENdpWf-chnciYD
width: 3468 pixels
height: 4624  pixels
channels: 3


In [35]:
data.shape

(71, 16)

In [36]:
data.head()

,Depression,Anxiety,Stress,Age,Gender,Azimuth Angle,Baseline,Space between Words,I-dot position,Slant,Loop,Average Size of Letters,Zone 1,Zone 2,Zone 3,Loop Sharpness
0,normal,normal,normal,18,Female,140,-41.025207,212.50,0,1.615385,1,5768.267442,158.203813,143.574359,159.457436,3.937316
1,moderate,extremely severe,moderate,19,Female,20,-44.998803,71.75,0,2.625001,1,14850.857143,194.916758,183.546506,202.464825,3.557768
2,normal,mild,normal,17,Female,25,-1.109084,157.25,0,-1.837500,0,11853.160000,159.064728,145.058776,164.177896,0.000000
3,normal,moderate,normal,21,Female,45,-0.017481,0.00,0,44.299999,1,888.712530,140.623952,152.637596,137.874585,2.897346
4,normal,mild,normal,22,Female,140,-32.121808,18.75,0,-3.636363,1,2436.885697,150.209401,154.170226,131.815745,1.191699


In [37]:
data.to_csv('processed_data.csv')